## Audiobooks - Preprocessing

### 1. Load libraries and data

In [1]:
import numpy as np
from sklearn import preprocessing

#Load the data
raw_data = np.loadtxt('Audiobooks_data.csv',delimiter = ',')

# unscaled inputs excluding customer_id and target
unscaled_inputs_all = raw_data[:,1:-1]

#targets are in the last column
targets_all = raw_data[:,-1]

In [2]:
len(targets_all)

14084

### 2. Preprocessing

#### 2.1. Balance the dataset

In [3]:
# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets = int(np.sum(targets_all))
num_one_targets

2237

There is a huge different between the number of 1 (customer did convert) and 0 (customer did not convert). Therefore, this dataset must be balanced to achieve better model accuracy.

In [4]:
# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0

# In order to create a balance dataset, some inputs/ targets have to be removed.
# # Declare a variable that will be removed:
indices_to_remove = []

# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(len(targets_all)):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Remove all indices that we marked "to remove" in the loop above.
unscaled_inputs_balanced = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_balanced = np.delete(targets_all, indices_to_remove, axis=0)

#### 2.2. Standardize the inputs

In [5]:
scaled_inputs = preprocessing.scale(unscaled_inputs_balanced)

#### 2.3. Shuffle the data

In [6]:
# Data was arranged by date, so since we will be batching, data should be shuffled by indices to be as randomly spread out as possible 
shuffled_indices = np.arange(len(scaled_inputs))
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_balanced[shuffled_indices]

#### 2.4. Split the dataset into train, validation, and test

In [7]:
# Count the total number of samples
samples_count = len(shuffled_inputs)

# the number of records in train, validation, and test subset, given a 80-10-10 distribution
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = int(0.1 * samples_count)

# Create variables that record the inputs and targets for training
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too.
# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1795.0 3579 0.5015367421067337
210.0 447 0.4697986577181208
232.0 447 0.5190156599552572


#### 2.5. Save datasets in *.npz

In [8]:
# Save the three datasets in *.npz.
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)